In [1]:
# 检测ollama是否正在运行
# 运行ollama: ollama run llama3
import psutil 

def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Lanunch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [13]:
# 通过REST api访问ollama
import urllib.request
import json

def query_model(
        prompt,
        model="llama3",
        url="http://localhost:11434/api/chat",
        role="user"
):
    # 构造请求数据
    data = {
        "model": model,
        "messages": [
            {"role": role, "content": prompt}
        ],
        "options": {
            # "seed": 123,
            "temperature": 1.,
            "top_p": 1,
            "num_ctx": 2048
        }
    }

    # dict转化json并编码
    payload = json.dumps(data).encode("utf-8")
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # 解析返回结果
    response_data = ""
    with urllib.request.urlopen(request) as response:
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]
    
    return response_data

model = "llama3"
result = query_model("What do Llamas eat?", model)
print(result)

Llamas are ruminant animals, which means they have a four-chambered stomach and primarily feed on plant-based foods. Adult llamas are herbivores and their diet typically consists of:

1. Grasses: Mowed grass, hay, or pasture grass.
2. Hay: Various types of hay such as alfalfa, timothy, or oat hay.
3. Grains: Oats, corn, and barley grains in moderation.
4. Leaves and shrubs: Llamas enjoy the leaves, twigs, and shrubs from trees and bushes.

In terms of nutrient requirements, llamas need a diet that provides:

* High-quality fiber (like hay) for digestive health
* Adequate protein (about 1-2% of their body weight) for growth and maintenance
* Limited amounts of energy-rich grains to avoid overfeeding

Here are some specific guidelines for feeding llamas:

* Hay: Provide unlimited access to good quality hay, making up about 80-90% of their diet.
* Grains: Limit grain intake to no more than 1-2 pounds per day (about 10-20% of their daily ration).
* Fresh water: Ensure access to clean, fres

## 提取指令数据

In [4]:
def extract_instruction(text):
    for content in text.split("\n"):
        if content:
            return content.strip()

In [8]:
# 使用一个特殊的query来提取指令
query = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>"

result = query_model(query, role="assistant")
instruction = extract_instruction(result)
print(instruction)

I have a small problem with my cat, Luna. She's been having some issues with her digestive system lately.


In [9]:
# 使用指令生成响应(回答)
response = query_model(instruction, role="user")
print(response)

Sorry to hear that Luna is experiencing some tummy troubles! Cats can be notoriously finicky about their food and habits, so it's not uncommon for them to have occasional digestive issues. Can you tell me a bit more about what's been going on? Has she been vomiting, having accidents outside the litter box, or showing any other symptoms that might give us clues to what's causing the problem?

Also, has her diet changed recently? Is she eating the same food and frequency as before, or have there been any changes in her eating habits? Any other details you can provide will help me better understand what might be going on with Luna.


In [16]:
# 生成数据集
from tqdm import tqdm 

dataset_size = 5
dataset = []

for i in tqdm(range(dataset_size)):
    result = query_model(query, role="assistant")
    instruction = extract_instruction(result)
    response = query_model(instruction, role="user")
    entry = {
        "instruction": instruction,
        "output": response
    }
    dataset.append(entry)

100%|██████████| 5/5 [01:19<00:00, 15.80s/it]


In [17]:
with open("instruction-data-llama3-7b.json", "w") as file:
    json.dump(dataset, file, indent=4)

In [18]:
!cat instruction-data-llama3-7b.json

[
    {
        "instruction": "What are some good alternatives to social media platforms?",
        "output": "There are many great alternatives to social media platforms that can help you maintain a healthy online presence, reduce distractions, and increase productivity. Here are some popular options:\n\n1. **Diaspora**: A decentralized, open-source social network that allows users to control their data.\n2. **Mastodon**: A fediverse platform that enables users to register on different servers (instances) while maintaining compatibility with other Mastodon instances.\n3. **PixelFed**: An image-sharing platform that focuses on sharing art, photography, and other creative work.\n4. **Discord**: A voice chat app primarily used for gaming communities, but also suitable for smaller social groups or online discussions.\n5. **Telegram**: A popular messaging app with group chats and the ability to create channels for sharing information.\n6. **Matrix**: An open-source, decentralized platform